# Retrieval

This notebook covers basic walkthrough of retrieval functionality in LangChain. For more information, see:

- [Retrieval Documentation](https://python.langchain.com/docs/modules/data_connection/)

- [Advanced Retrieval Types](https://python.langchain.com/docs/modules/data_connection/retrievers/)

- [QA with RAG Use Case Documentation](https://python.langchain.com/docs/use_cases/question_answering/)

## Load Documents

In [1]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [2]:
len(docs)

1

## Split documents

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [4]:
len(documents)

5

## Index Documents

In [12]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
vector = FAISS.from_documents(documents, embeddings)

## Query Documents


In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")
llm = ChatOpenAI()

document_chain = create_stuff_documents_chain(llm, prompt)

In [14]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [15]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by providing a way to run chains over datasets and visualize the outputs. It simplifies dataset uploading and allows users to review and evaluate the results. The LangSmith client makes it easy to pull down a dataset and run a chain over the data points, logging the results to a new project associated with the dataset. Users can assign feedback to runs and mark them as correct or incorrect directly in the web app, displaying aggregate statistics for each test project. LangSmith also provides evaluators that can be specified when initiating a test run to evaluate the results. Additionally, LangSmith offers annotation queues for manual review and annotation of runs, allowing users to assess subjective qualities and validate auto-evaluated runs.


## Advanced Retrieval

In [16]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=llm)

In [17]:
retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [18]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

/Users/harrisonchase/.pyenv/versions/3.11.1/envs/langchain-0.1-guides/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


LangSmith can help with testing by providing the following features:

1. Visualization of Inputs/Outputs: LangSmith provides a straightforward visualization of the exact inputs and outputs to all LLM calls, allowing users to easily understand them.

2. Prompt Editing Playground: LangSmith has a built-in playground where users can modify the prompt of an LLM call and re-run it to observe the resulting changes to the output. This feature supports OpenAI and Anthropic models and works for LLM and Chat Model calls.

3. Sequence of Events: LangSmith's tracing feature offers a visualization of the sequence of calls and interactions in complicated chains and agents. This helps users understand the exact sequence of events and the inputs/outputs of each call.

4. Latency Monitoring: LangSmith tracks the latency of each step in a chain, allowing users to identify and potentially eliminate the slowest components that may be causing a chain to take longer than expected.

5. Token Usage Tracking: 